In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

In [ ]:
#File Path

file_path= '../input/credit-card-customers/BankChurners.csv'

data = pd.read_csv(file_path, index_col='CLIENTNUM')

data = data.drop('Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', axis=1)
data = data.drop('Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2', axis=1)

data.head()

In [ ]:
data["Attrition_Flag"].value_counts()

In [ ]:
data.info()

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
data.hist(bins=10)
plt.show()

Almost all of the histograms are skewed , since the data is not well distributed, probably the skewed features should be dealt as categorical data

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
data["encoded_Attrition_Flag"] = label_encoder.fit_transform(data["Attrition_Flag"])

In [ ]:
data_num_corr = data_num.corr()
data_num_corr

In [ ]:
# dropping very low correlations 
#dropping Attrition_Flag column because it has been considered in encoded_attrition_flag
data.drop(["Avg_Open_To_Buy","Attrition_Flag"], axis=1, inplace=True)


In [ ]:
data_num = data.drop(["Gender","Education_Level","Marital_Status","Income_Category","Card_Category"], axis = 1)

In [ ]:
data.isnull().sum()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
#Dealing with Categorical Data

#Encoding the target variable y

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder

y= data['encoded_Attrition_Flag'] 


num_attrib = list(data_num)
ordinal_attrib = ["Education_Level","Income_Category","Card_Category"]
one_hot_attrib = ["Gender", "Marital_Status"]

full_pipeline = ColumnTransformer([
    ("num",MinMaxScaler(),num_attrib),
    ("ordinal",OrdinalEncoder(),ordinal_attrib),
    ("one_hot",OneHotEncoder(),one_hot_attrib)
])
data_prepared = full_pipeline.fit_transform(data)
data.drop(["encoded_Attrition_Flag"], axis=1 , inplace = True)

X= data # X is same as data


In [ ]:
X = data_prepared

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

# Function for comparing different approaches
model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(X_train, y_train)
preds = model.predict(X_valid)
print(preds)
print(y_valid)
accuracy= accuracy_score(y_valid, preds)*100
print("Accuracy is: ", accuracy, "%")
# Save test predictions to file
# output = pd.DataFrame({'Id': imputed_X_valid.index,
#                        'Attrition prediction': preds,
#                        'actual_y_valid': label_y_valid})
# output.to_csv('submission.csv', index=False)
# print(output)

In [ ]:
from sklearn.model_selection import cross_val_predict
y_train_pred = cross_val_predict(model, X_train,y_train, cv=3)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,y_train_pred)

In [ ]:
y_valid.reset_index(drop=True, inplace=True)

In [ ]:
# Save test predictions to file
output = pd.DataFrame({'Attrition prediction': preds,
                       'actual_y_valid': y_valid})
output.to_csv('submission.csv', index=False)
print(output)